In [ ]:
import torch.nn as nn
import pandas as pd

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

## Load Dataset

In [ ]:
math_dataset = pd.read_csv("../data/diverse_math_questions.csv")
math_dataset

,Question,Answer
0,What is 16 × 18?,288
1,What is the square root of 81?,9
2,"If a square has a side length of 8, what is it...",64
3,What is 71 + 84?,155
4,What is 85 + 12?,97
5,What is the square root of 324?,18
6,What is 18 × 12?,216
7,What is 10 + 72?,82
8,What is 78 - 20?,58
9,What is 39 ÷ 3?,13


## Load Model


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "meta-llama/Llama-3.2-1B-Instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"

# Ensure torch_dtype is set to torch.bfloat16
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16  # Use bfloat16 for better memory efficiency
).to(device)

tokenizer = AutoTokenizer.from_pretrained(model_id)
if getattr(tokenizer, "pad_token", None) is None:
    tokenizer.pad_token = tokenizer.eos_token

## Fine-Tuning

In [ ]:
!pip install transformers accelerate trl

In [ ]:
from transformers import TrainingArguments
import datasets
from trl import SFTTrainer
from utils.utils import formatting_prompts_func, run_model_qa

training_args = TrainingArguments(
    report_to="none",
    learning_rate=5e-5,
    lr_scheduler_type="constant_with_warmup",
    warmup_steps=10,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    output_dir="output_model",
    overwrite_output_dir=True,
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=False,
    logging_steps=1,
    seed=0,
    do_train=True,
    do_eval=True,
    do_predict=False,
    eval_strategy="epoch",
    gradient_checkpointing=True,
    max_grad_norm=0.3,
    push_to_hub=False,
    hub_private_repo=True,
)

trainer = SFTTrainer(
    model = model,
    args=training_args,
    train_dataset=datasets.Dataset.from_pandas(math_dataset[:40]),
    eval_dataset=datasets.Dataset.from_pandas(math_dataset[40:]),
    packing=False,
    tokenizer=tokenizer,
    max_seq_length=500,
    formatting_func=formatting_prompts_func,
)

trainer.train()

response_template: 
 ### Answer: 


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,1.229300,1.144131
2,0.825500,0.973357
3,0.373100,0.853033


TrainOutput(global_step=15, training_loss=1.2791778186957041, metrics={'train_runtime': 375.563, 'train_samples_per_second': 0.32, 'train_steps_per_second': 0.04, 'total_flos': 16208721641472.0, 'train_loss': 1.2791778186957041, 'epoch': 3.0})

In [ ]:
test_qa = "### Question: What is 12 × 20\n"
assistant_response = run_model_qa(model=model, tokenizer=tokenizer, messages=test_qa, max_new_tokens=100, verbose=False)
print(f"\n###Assistant response:###\n{assistant_response}")

test_qa = "### Question: What is 16 × 18\n"
assistant_response = run_model_qa(model=model, tokenizer=tokenizer, messages=test_qa, max_new_tokens=100, verbose=False)
print(f"\n###Assistant response:###\n{assistant_response}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_m


###Assistant response:###
### Answer: 240
 ### Answer: 18 × 14
 ### Answer: 36 ÷ 18
 ### Answer: 2
 ### Answer: 18 ÷ 6
 ### Answer: 3
 ### Answer: 18 ÷ 9
 ### Answer: 2

 ### Answer: 18 ÷ 18
 ### Answer: 1
 ### Answer: 18 ÷ 1
 ### Answer: 18 ÷ 18
 ###

###Assistant response:###
### Answer: 288
 ### Answer: 18 × 16
 ### Answer: 288

 ### Answer: 18 × 16
 ### Answer: 288
 ### Answer: 18 × 16
 ### Answer: 288
 ### Answer: 18 × 16
 ### Answer: 288
 ### Answer: 18 × 16
 ### Answer: 288
 ### Answer: 18 × 16
 ### Answer: 288
 ### Answer:
